In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

import numpy as np

import My_dataset as md
from preprocess import build_vocab, tokenize, numericalize, pad_sequences, TranslationDataset
import building_blocks as bb
import embeddings as emb
import tools

import os
from torch import save as torch_save
from torch import load as torch_load

print("Starting main.py file ...")

Starting main.py file ...


In [2]:
# Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EMBED_DIM = 300
MAX_LENGTH = 20

print("I will be loading the embeddings ...")

# Pre-trained embeddings
en_emb = emb.load_embeddings('en')
de_emb = emb.load_embeddings('de')

# Tokenization and Vocabulary
print("will start tockenizing ...")

def build_vocab(sentences, min_freq=1):
    vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
    word_counts = {}
    for sent in sentences:
        for word in sent.lower().split():
            word_counts[word] = word_counts.get(word, 0) + 1
    
    idx = 4
    for word, count in word_counts.items():
        if count >= min_freq:
            vocab[word] = idx
            idx += 1
    return vocab

I will be loading the embeddings ...
I will be importing the en embeddings now / -> from embeddings.py file
I will be importing the de embeddings now / -> from embeddings.py file
will start tockenizing ...


In [3]:

# Build vocabularies
src_sentences = [pair[0] for pair in md.sampled_pairs]
trg_sentences = [pair[1] for pair in md.sampled_pairs]

src_vocab = build_vocab(src_sentences)
trg_vocab = build_vocab(trg_sentences)

# Create embedding matrices
src_embed_matrix = emb.create_embedding_matrix(src_vocab, en_emb)
trg_embed_matrix = emb.create_embedding_matrix(trg_vocab, de_emb)


In [4]:
print("will initialize the model...")

# Initialize model
model = bb.Transformer(
    src_vocab_size=len(src_vocab),
    trg_vocab_size=len(trg_vocab),
    src_pad_idx=0,
    trg_pad_idx=0,
    embed_size=EMBED_DIM,
    num_layers=2,
    heads=4,
    forward_expansion=4,
    dropout=0.1,
    device=DEVICE,
    max_length=MAX_LENGTH
)

will initialize the model...


In [6]:
BATCH_SIZE = 12
NUM_EPOCHS = 400

In [7]:

# Replace embeddings with pre-trained
model.encoder.word_embedding.weight.data.copy_(src_embed_matrix)
model.decoder.word_embedding.weight.data.copy_(trg_embed_matrix)

# Freeze embeddings if needed
# model.encoder.word_embedding.weight.requires_grad = False
# model.decoder.word_embedding.weight.requires_grad = False

model = model.to(DEVICE)

# Dataset and DataLoader
dataset = TranslationDataset(md.sentence_pairs, src_vocab, trg_vocab, MAX_LENGTH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


In [8]:

# Training setup
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:


print("Starting the training loop ...")

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    progress_bar = tqdm(loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    
    for src, trg in loader:
        src = src.to(DEVICE)
        trg = trg.to(DEVICE)
        
        # Forward pass
        optimizer.zero_grad()
        output = model(src, trg[:, :-1])
        output_dim = output.shape[-1]
        
        # Compute loss
        loss = criterion(output.reshape(-1, output_dim), trg[:, 1:].reshape(-1))
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(loader)
    print(f'Epoch {epoch+1:03d} | Loss: {avg_loss:.4f}')
    # Checkpointing
    # Save checkpoint every 50 epochs
    if (epoch+1) % 50 == 0:
        checkpoint_path = f"checkpoints/model_epoch_{epoch+1}.pth"
        torch_save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'src_vocab': src_vocab,
            'trg_vocab': trg_vocab,
            'max_length': MAX_LENGTH
        }, checkpoint_path)
        #torch_save(checkpoint, f"checkpoints/model_epoch_{epoch+1}.pth")
        print(f"Checkpoint saved for epoch {epoch+1} at {checkpoint_path}")


Starting the training loop ...


Epoch 1/400:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 001 | Loss: 4.6778


Epoch 1/400:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 002 | Loss: 4.3140


...

Epoch 398 | Loss: 0.0373


Epoch 398/400:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 399 | Loss: 0.0324


Epoch 399/400:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 400 | Loss: 0.0331
Checkpoint saved for epoch 400 at checkpoints/model_epoch_400.pth


In [ ]:
# # Create checkpoints directory if it doesn't exist
# os.makedirs("checkpoints", exist_ok=True)

In [12]:
test_model, src_vocab, trg_vocab = tools.load_test_model(
    checkpoint_path,
    TransformerClass=bb.Transformer,
    embed_size=EMBED_DIM,
    device=DEVICE,
    max_length=MAX_LENGTH
)

✅ Model loaded successfully from checkpoints/model_epoch_400.pth


d:\projects\Deep Learning\transformers\tools.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


In [13]:

print("testing translation ...")

# Test translation
test_sentence = "good morning"
print(f'Input: {test_sentence}')
print(f'Output: {tools.translate(test_sentence, test_model, src_vocab, trg_vocab, DEVICE)}')

testing translation ...
Input: good morning
Output: guten morgen.
